In [8]:
# Fist install the library

#%pip install aepy

In [9]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np

from sklearn.metrics import accuracy_score
from keras import utils

from rapidae.data.datasets import load_MNIST
from rapidae.data.utils import evaluate, display_diff, add_noise
from rapidae.models.ae.ae_model import AE
from rapidae.models.base.default_architectures import VanillaEncoder, VanillaDecoder
from rapidae.pipelines.training import TrainingPipeline

Download and preprocess the dataset. In this example, the selected dataset is the well-known MNIST composed of handwritten number images.

In [10]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-20 19:26:02 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-20 19:26:02 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-20 19:26:02 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-20 19:26:02 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


Since we want to train a denoising autoencoder, we also need to add some noise to the images.
The noise factor should be a float between 0 and 1.

In [11]:
# Add noise to the train and test data
x_train_noisy = add_noise(x_train, noise_factor=0.4)
x_test_noisy = add_noise(x_test, noise_factor=0.4)

x_train = x_train.reshape(x_train.shape[0], -1)
x_train_noisy = x_train_noisy.reshape(x_train_noisy.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
x_test_noisy = x_test_noisy.reshape(x_test_noisy.shape[0], -1)

Compose the dictionaries to feed the autoencoder during training and evaluation phase.
Note in train how the train labels correspond to the original MNIST images (without noise).

In [12]:
train_data = dict(data=x_train_noisy.astype(float), labels=x_train)
test_data = dict(data=x_test_noisy.astype(float), labels=y_test)

Since the denoising autoencoder in structure is a normal autoencoder (only its functionality varies), the encoder and decoder are the vanilla ones, you can specify the depth and number of neurons per layer in each using the layers_conf parameter.

In [13]:
# Model creation
model = AE(input_dim=x_train_noisy.shape[1], latent_dim=2,
           encoder=VanillaEncoder, decoder=VanillaDecoder, layers_conf=[64, 32])

Define the training pipeline. There you can fix some hyperparameters realted to the training phase of the autoencoder, like learning rate, bath size, numer of epochs, etc

In [14]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=10)

trained_model = pipe(train_data)

2023-12-20 19:26:03 [INFO]: +++ training_pipeline +++
2023-12-20 19:26:03 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-20_19-26-03
2023-12-20 19:26:03.830569: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 19:26:03.915676: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 19:26:03.915864: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be a

Epoch 1/10


/home/lucaspc/venvs/keras_core/lib/python3.11/site-packages/keras/src/layers/layer.py:1217: UserWarning: Layer 'encoder' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encoutered: ''Layer 'dense_6' expected 1 input(s). Received 2 instead.''
  warnings.warn(
/home/lucaspc/venvs/keras_core/lib/python3.11/site-packages/keras/src/layers/layer.py:358: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it look

ValueError: Exception encountered when calling VanillaEncoder.call().

[1mLayer 'dense_6' expected 1 input(s). Received 2 instead.[0m

Arguments received by VanillaEncoder.call():
  • x={'data': 'tf.Tensor(shape=(128, 784), dtype=float32)', 'labels': 'tf.Tensor(shape=(128, 784), dtype=uint8)'}

Evaluation phase using test data

In [ ]:
y_hat = trained_model.predict(test_data)

Finally the original images with noise can be graphically compared with the output images of the autoencoder.

In [ ]:
display_diff(x_test_noisy, y_hat['recon'])